In [1]:
import sys, os
sys.path.append('../MultiFishLSS/')
from headers import *
from twoPoint import *
from twoPointNoise import *
from classy import Class


## Generate BAO forecasts

#### Fiducial cosmology

In [5]:
default_cosmo = {
        'output': 'tCl lCl mPk',
        'non linear':'halofit',
        'l_max_scalars': 4000,
        'lensing': 'yes',
        'P_k_max_h/Mpc': 2.,
        'z_pk': '0.0,1087',
        'A_s': 2.10732e-9,
        'n_s': 0.96824,
        'alpha_s': 0.,
        'h': 0.6736,
        'N_ur': 2.0328,
        'N_ncdm': 1,
        'm_ncdm': 0.06,
        'tau_reio': 0.0544,
        'omega_b': 0.02237,
        'omega_cdm': 0.1200,
        'Omega_k': 0.,
        'Omega_Lambda': 0.,
        'w0_fld':-1,
        'wa_fld':0}
cosmo = Class()
cosmo.set(default_cosmo)
cosmo.compute()
h = cosmo.h()

chi = lambda zz: cosmo.comoving_distance(zz)*h

#### Experiment configuration

In [6]:
fsky = 5000/41253 # 5000 deg2

# two redshift bins from z = 2.7 to 3.3
zmin, zmax = 2.7, 3.3
nbins = 2

# 2 galaxy samples 
# name, bias, number density as lists
samples=['ga','gb']
b = [lambda z: 2.5, lambda z: 3.5] 
n = [lambda z: 2e-4, lambda z: 2e-4]

# overlap between stochastic terms
# index 0 and 2 are auto-correlations, so they must be 1
# index 1 is the cross-correlation between the two samples, set to 0 by default for no overlap
fover=[1,0,1]

# experiment name and basedir
bd='./'
surveyname='example'

exp = experiment(zmin=zmin, zmax=zmax, nbins=nbins, fsky=fsky, b=b, n=n,samples=samples,fover=fover)
forecast = fisherForecast(experiment=exp,cosmo=cosmo,name=surveyname,basedir=bd,ell=np.arange(10,3000,1))    

#### Compute derivatives (if not done already)

Derivatives are computed faster with dedicated node, using `example_setup.sh`

Turn `forecast.recon` to `True` to work on BAOs

In [11]:
basis = np.array(['alpha_perp','alpha_parallel','b'])

# set recon = True, so that we perform BAO reconstruction when computing the power spectrum
forecast.recon = True

forecast.free_params = basis
forecast.compute_derivatives() # will not overwrite unless specified

#### Use derivatives to construct Fisher matrix and get forecasts

In [12]:
derivs = forecast.load_derivatives(basis)
F = lambda i: forecast.gen_fisher(basis, 100, derivatives=derivs, zbins=np.array([i]))
Fs = [F(i) for i in range(nbins)]
Fs = np.array(Fs)

In [13]:
Finvs = [np.linalg.inv(Fs[i]) for i in range(nbins)]
saperp = [np.sqrt(Finvs[i][0,0]) for i in range(nbins)]
saparr = [np.sqrt(Finvs[i][1,1]) for i in range(nbins)]
print('Relative error on DA/rd:',saperp)
print('Relative error on H*rd:',saparr)


Relative error on DA/rd: [0.007821436282346966, 0.008327988194172964]
Relative error on H*rd: [0.011513224417099776, 0.012052890249509144]


In [14]:
forecast.recon = False


Fixed shape $\sigma_8$ (assumes cross-correlation with SO)

In [13]:
basis = np.array(['log(A_s)','b','N',])
# derivatives = forecast.load_derivatives(basis)
# F = forecast.gen_fisher(basis,7,kmax_knl=1.,derivatives=derivatives)

for i in range(nbins):
    globe = 1
    F = forecast.gen_lensing_fisher(basis,globe,kmax_knl=.2,bins=np.array([i]),kk=False,no_kg=False) 
    idxs = np.arange(globe).tolist()
    for j in range(F.shape[0]-globe):
        if j%nbins == i: idxs += [globe + j]
    Finv = np.linalg.inv(F[idxs][:,idxs])
    print('(sigma s8)/s8 at z=%.2f: %.2e'%(forecast.experiment.zcenters[i],np.sqrt(np.diag(Finv))[0]/2) )# ds8/s8 

[1, 3, 5, 7, 9]
(sigma s8)/s8 at z=2.86: 4.04e-02
[2, 4, 6, 8, 10]
(sigma s8)/s8 at z=3.19: 3.74e-02


In [ ]:
basis = np.array(['log(A_s)','b','N',])
F = forecast.gen_lensing_fisher(basis,1,kmax_knl=.2) 
Finv = np.linalg.inv(F)
print('sigma s8: %.2e'%(np.sqrt(np.diag(Finv))[0]/2) ) # ds8/s8 